In [77]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

openai.api_type = "azure"
openai.api_base = "https://cog-oaxatpknic6wq.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "a4afc38d3a6246d3b855a8adcca0d0f1"

#endpoint = "https://gptkb-oaxatpknic6wq.search.windows.net"

model_name = "gpt-35-turbo"
deployment_name ="chat"
prompt = "Who is Charles Babbage?"

azure_credential = DefaultAzureCredential()

In [78]:
knowledge_base = {
    "source1.txt" : "data\Azure HDInsight for Apache Spark 3.3 is now available for public preview _ Azure updates _ Microsoft Azure.html",
    "source2.txt" : "data\Azure VMware Solution now available in North Switzerland _ Azure updates _ Microsoft Azure.html",
    "source3.txt" : "data\General availability_ Azure HX Virtual Machines for HPC _ Azure updates _ Microsoft Azure.html",
    "source4.txt" : "data\General availability_ Query performance insights for Azure Database for PostgreSQL – Flexible Server _ Azure updates _ Microsoft Azure.html",
    "source5.txt" : "data\Microsoft Azure Load Testing - additional Azure components for server-side monitoring _ Azure updates _ Microsoft Azure.html",
    "source6.txt" : "data\Public preview_ Add-on and node image in AKS release tracker _ Azure updates _ Microsoft Azure.html",
    "source7.txt" : "data\Public preview_ Assess impact of service retirements workbook template in Azure Advisor _ Azure updates _ Microsoft Azure.html",
    "source8.txt" : "data\Public Preview_ Azure Virtual Desktop Insights Powered by the Azure Monitor Agent _ Azure updates _ Microsoft Azure.html",
    "source9.txt" : "data\Public preview_ Confidential Virtual Machines (VM) support in Azure Virtual Desktop _ Azure updates _ Microsoft Azure.html",
    "source10.txt" : "data\Public preview_ Custom Image Templates for Azure Virtual Desktop _ Azure updates _ Microsoft Azure.html",
    "source11.txt" : "data\Public Preview_ GraphQL resolvers for Azure Cosmos DB, Azure SQL in Azure API Management _ Azure updates _ Microsoft Azure.html",
}



# ChatGPT uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
Assistant helps the company employees with their Microsoft Azure questions.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question. 
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].


Sources:
[source1.txt][source2.txt][source3.txt][source4.txt][source5.txt][source6.txt][source7.txt][source8.txt][source9.txt][source10.txt][source11.txt]

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question. Source names are not good search terms to include in the search query.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [79]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "Who is Charles Babbage?"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

if len(history) > 0:
    completion = openai.Completion.create(
        prompt = prompt ,
        max_tokens = 10,
        temperature = 0.9,
        engine=deployment_name)
    search = completion.choices[0].text
else:
    search = user_input

# Alternatively simply use search_client.search(q, top=3) if not using semantic search
print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

completion = openai.Completion.create(
    prompt = prompt ,
    max_tokens = 100,
    temperature = 0,
    engine=deployment_name,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))
print("\n-------------------\nPrompt:\n" + prompt)

#print(completion)

Searching: Who is Charles Babbage?
-------------------
user: Who is Charles Babbage?
-------------------
assistant:  Charles Babbage was an English mathematician and inventor who is credited with inventing the first mechanical computer that eventually led to more complex designs. He was born on December 26, 1791, in London, England, and died on October 18, 1871, in Marylebone, London, England. Babbage was a polymath who made significant contributions to various fields, including mathematics, engineering, and computer science. He is best known for his work on the Analytical Engine

-------------------
Prompt:
Who is Charles Babbage?
